In [42]:
from pprint import pprint
import pandas as pd
import numpy as np
# import matplotlib.py ploy as plt
# import seaborn as sns

In [43]:
import praw

user_agent = "Scraper 1.0 by /u/danieljagun"
reddit = praw.Reddit(
    client_id="K-AfR5cf3qvXEXM_MTn7IQ",
    client_secret="JdAlNMb6dPkWCd4bX4PdXCwIpbA5gQ",
    user_agent=user_agent
)

# hot new rising top
headlines = set()
for submission in reddit.subreddit('CryptoCurrency').hot(limit=None):
    print(submission.title)
    print(submission.id)
    print(submission.author)
    print(submission.created_utc)
    print(submission.score)
    print(submission.upvote_ratio)
    print(submission.url)
    break
    headlines.add(submission.title)
print(len(headlines))

In [44]:
# hot new rising top
headlines = set()
for submission in reddit.subreddit('CryptoCurrency').hot(limit=None):
    headlines.add(submission.title)
print(len(headlines))

331


In [45]:
df = pd.DataFrame(headlines)
df.head()

,0
0,The Great Wall in Web3 (Research)
1,Binance Says It Has Onboarded New Euro Fiat Pa...
2,Syntropy's Data Layer Whitepaper
3,"SEC's Bitcoin ETF Decision Today - VanEck, Wis..."
4,"Morgan Stanley believes crypto winter is over,..."


In [46]:
df.to_csv('titles.csv', header=False, encoding='utf-8', index=False)

In [47]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/daniel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [48]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SENIA

senia = SENIA()
results = []

for line in headlines:
    pol_score = senia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

pprint(results[:3], width=100)

[{'compound': 0.6249,
  'headline': 'The Great Wall in Web3 (Research)',
  'neg': 0.0,
  'neu': 0.549,
  'pos': 0.451},
 {'compound': 0.0,
  'headline': 'Binance Says It Has Onboarded New Euro Fiat Partners for Deposits, Withdrawals',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.0,
  'headline': "Syntropy's Data Layer Whitepaper",
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0}]


In [49]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.0,0.549,0.451,0.6249,The Great Wall in Web3 (Research)
1,0.0,1.000,0.000,0.0000,Binance Says It Has Onboarded New Euro Fiat Pa...
2,0.0,1.000,0.000,0.0000,Syntropy's Data Layer Whitepaper
3,0.0,1.000,0.000,0.0000,"SEC's Bitcoin ETF Decision Today - VanEck, Wis..."
4,0.0,1.000,0.000,0.0000,"Morgan Stanley believes crypto winter is over,..."


In [50]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.0,0.549,0.451,0.6249,The Great Wall in Web3 (Research),1
1,0.0,1.000,0.000,0.0000,Binance Says It Has Onboarded New Euro Fiat Pa...,0
2,0.0,1.000,0.000,0.0000,Syntropy's Data Layer Whitepaper,0
3,0.0,1.000,0.000,0.0000,"SEC's Bitcoin ETF Decision Today - VanEck, Wis...",0
4,0.0,1.000,0.000,0.0000,"Morgan Stanley believes crypto winter is over,...",0


In [51]:
df2 = df[['headline', 'label']]

In [52]:
df2.to_csv('reddit_title_label.csv', encoding='utf-8', index=False)

In [53]:
df.label.value_counts()

label
 0    169
 1     98
-1     64
Name: count, dtype: int64

In [54]:
print("Positive Titles:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)

print("Negative Titles:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)

Positive Titles:

['The Great Wall in Web3 (Research)',
 'Blockchain improves charity transparency — But is it right for everyone?',
 'Amid Rising Bitcoin ETF Hopes, Critics Warn of Vulnerability and Echoes of Precious Metal Manipulation – Bitcoin News',
 'GPUtopia is a marketplace where you can very easily sell your GPU computing power to train AI models and be paid back in Bitcoin. Login on thier site with an Alby wallet (lightning network), click '
 'on "load model" and your good to go. ChatGPU is their decentralized GPT.',
 'Stellar, Early Blockchain Built for Payments, Adds Smart Contracts to Take on Ethereum']
Negative Titles:

['Medical Doctor Confesses to Paying Hitman $25K in Bitcoin to Kill His Girlfriend',
 'Solana News: $122 Million in Solana Projected to be Sold as Part of the Dump has been Staked',
 'Sugar Land man, 37-year-old Xiaofei Chen, pleads guilty to identity theft in $500,000 Bitstamp cryptocurrency fraud scheme',
 'Coinbase picks Ireland as its main EU regulator